# 3. Bayesian Optimization

Learn how to write a Bayesian optimization script to intelligently adjust RGB power levels to match a target color and visualize optimization results.

<img src="../../_static/sdl-demo/grid-vs-random-vs-bayesian-ax-logo.png" width=80%>

*An optimization efficiency comparison between grid search, random search, and Bayesian optimization using Meta's Ax Platform, where Bayesian optimization is the most efficient at matching the target color.*

## A Gentle Introduction to Bayesian Optimization

Choosing the right optimization method depends on the specific needs and constraints of a project.

✅ Watch the <a href="https://youtu.be/IVaWl2tL06c?si=d1uADy-Xxcj4JD_G" target="_blank">Gentle Introduction to Bayesian Optimization</a> below. This one is very important to understand the basics of Bayesian optimization and how it compares to traditional design of experiments.

<div style="position: relative; overflow: hidden; padding-top: 50%; margin-bottom: 100px; width: 75%;">
    <iframe src="https://www.youtube.com/embed/IVaWl2tL06c?si=iSmEFYLJhXIXJam6" title="YouTube video player" style="position: absolute; top: 0; left: 0; width: 100%; height: 100%; border: 0;" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share; fullscreen" allowfullscreen="allowfullscreen" mozallowfullscreen="mozallowfullscreen" msallowfullscreen="msallowfullscreen"  oallowfullscreen="oallowfullscreen" webkitallowfullscreen="webkitallowfullscreen"></iframe>
</div>

✅ Watch <a href="https://youtu.be/HZGCoVF3YvM?si=Gfoi-0sRvxqsTW2u" target="_blank">Bayes rule by 3Blue1Brown</a>. This will help you understand the most basic statistical concept that powers Bayesian optimization algorithms.

<div style="position: relative; overflow: hidden; padding-top: 50%; margin-bottom: 100px; width: 75%;">
    <iframe src="https://www.youtube.com/embed/HZGCoVF3YvM?si=Gfoi-0sRvxqsTW2u" title="YouTube video player" style="position: absolute; top: 0; left: 0; width: 100%; height: 100%; border: 0;" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share; fullscreen" allowfullscreen="allowfullscreen" mozallowfullscreen="mozallowfullscreen" msallowfullscreen="msallowfullscreen"  oallowfullscreen="oallowfullscreen" webkitallowfullscreen="webkitallowfullscreen"></iframe>
</div>

<!-- ✅ Review the scikit-learn documentation for [`ParameterGrid`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ParameterGrid.html) -->

✅ Read the *Why use quasi-random search?* section from [a quasi-random search guide](https://developers.google.com/machine-learning/guides/deep-learning-tuning-playbook/quasi-random-search)

✅ Review the scipy documentation for two types of quasi-random search: [`Sobol`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.qmc.Sobol.html) and [Latin Hypercube (LHE)](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.qmc.LatinHypercube.html)

✅ Spend 20 minutes exploring the [Ax Platform Docs](https://ax.dev/)

✅ Complete Ax's [Service API Tutorial](https://ax.dev/tutorials/gpei_hartmann_service.html) [[Colab link](./1.3.1-ax-service-api.ipynb)]

✅ Run the script below in a Python interpreter of your choice with the `ax-platform` dependency installed (i.e., `pip install ax-platform`). For convenience, you may use the Open in Colab badge in [this companion notebook](./1.3.2-ax-service-api-basic.ipynb).

In [1]:
import math
from ax.service.ax_client import AxClient, ObjectiveProperties

obj1_name = "branin"

def branin(x1, x2):
    y = float(
        (x2 - 5.1 / (4 * math.pi**2) * x1**2 + 5.0 / math.pi * x1 - 6.0) ** 2
        + 10 * (1 - 1.0 / (8 * math.pi)) * math.cos(x1)
        + 10
    )

    return y

ax_client = AxClient()
ax_client.create_experiment(
    parameters=[
        {"name": "x1", "type": "range", "bounds": [-5.0, 10.0]},
        {"name": "x2", "type": "range", "bounds": [0.0, 15.0]},
    ],
    objectives={
        obj1_name: ObjectiveProperties(minimize=True),
    },
)

for _ in range(10):
    parameters, trial_index = ax_client.get_next_trial()
    results = branin(
        parameters["x1"],
        parameters["x2"],
    )
    ax_client.complete_trial(trial_index=trial_index, raw_data=results)

best_parameters, metrics = ax_client.get_best_parameters()

[WARNING 01-05 18:54:05] ax.service.utils.with_db_settings_base: Ax currently requires a sqlalchemy version below 2.0. This will be addressed in a future release. Disabling SQL storage in Ax for now, if you would like to use SQL storage please install Ax with mysql extras via `pip install ax-platform[mysql]`.
[INFO 01-05 18:54:05] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.
[INFO 01-05 18:54:05] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter x1. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 01-05 18:54:05] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter x2. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool'

## Additional Resources

For the mathematically inclined or those who would like to understand the theory behind Bayesian optimization, you are encouraged to work through [A Visual Exploration of Gaussian Processes](https://distill.pub/2019/visual-exploration-gaussian-processes/), which will give you a better sense of what Gaussian processes are and how Bayesian inference is used to perform Gaussian process regression. Note that Gaussian processes are the most widely used surrogate model in Bayesian optimization applications. Next, you can work through [Exploring Bayesian Optimization](https://distill.pub/2020/bayesian-optimization/). These two resources will help you to develop intuition about Bayesian optimization by understanding the underlying theory.

You may also be interested in the following resources:

- [How are materials discovered?](https://youtu.be/RRNcqJSJ6vc?si=_sdNLUZFbW-3APrJ) YouTube video
- [Grid vs. random vs. Bayesian optimization](https://towardsdatascience.com/grid-search-vs-random-search-vs-bayesian-optimization-2e68f57c3c46) general machine learning tutorial (requires a Medium account)
- [Tuning the hyper-parameters of an estimator](https://scikit-learn.org/stable/modules/grid_search.html) scikit-learn tutorial
- [The Olympus benchmarking package](https://aspuru-guzik-group.github.io/olympus/classes/planners/index.html) with a variety of other optimization algorithms documented
- [A `self-driving-lab-demo` discussion](https://github.com/sparks-baird/self-driving-lab-demo/discussions/57) which has a list of optimization algorithms 
- [Honegumi](https://github.com/sgbaird/honegumi), an interactive "skeleton code" generator for optimization packages

## 

That's it! You've completed the tutorial 🎉. Return to the course website to do a knowledge check.